In [ ]:
import numpy as np
f = open('ADNI_TSV_FILE.raw', 'r')
def generate_raw_snp_samples(line):
    FID, IID, PAT,  MAT, SEX, SNPs = line[0], line[1], line[2], line[3], line[4], line[6:]
    return FID, IID, PAT,  MAT, SEX, np.array(SNPs).astype(int)

cnt = 0
missing = set()
while True:
    cnt += 1
    line = f.readline()
    line = line.split()
#     print(len(line[6:]));break
    temp = {i for i, x in enumerate(line) if x == "NA"}
    missing = missing | temp
    print(cnt, ':', len(temp))
    if len(line) == 0:
        break
# FID, IID, PAT,  MAT, SEX, SNPs = generate_raw_snp_samples(line=line)
# FID, IID, PAT,  MAT, SEX, SNPs[:10]
len(missing)
f.close()

# (1628272, 0.7635722458779614)

In [ ]:
len(missing), len(missing) / 2132440

In [ ]:
import numpy as np
f = open('ADNI_TSV_FILE.raw', 'r')

cnt = 0
missing = {}
while True:
    cnt += 1
    line = f.readline()
    line = line.split()
    for i, x in enumerate(line[6:]):
        if x == "NA":
            if i in missing:
                missing[i] += 1
            else:
                missing[i] = 1
    print(cnt, ':')
    if len(line) == 0:
        break
# FID, IID, PAT,  MAT, SEX, SNPs = generate_raw_snp_samples(line=line)
# FID, IID, PAT,  MAT, SEX, SNPs[:10]
len(missing)
f.close()

In [ ]:
missing2 = [[x, missing[x]] for x in missing]

In [ ]:
missing2 = sorted(missing2, key=lambda x:x[1], reverse=True)
missing2 = np.array(missing2)
missing2

In [ ]:
cnt2 =(missing2[:, 1] > 5).sum() 
(cnt2, cnt2 / 2132434), (5 / 847)
snps_to_consider = list(set(range(2132434)) - set(missing2[missing2[:, 1] > 5][:, 0]))
len(snps_to_consider), (missing2[:, 1] > 5).sum()

import pickle
pickle.dump(snps_to_consider, open('snps_to_consider.pkl', 'wb'))

In [ ]:
from sklearn.preprocessing import OneHotEncoder
enc = OneHotEncoder(categories='auto', handle_unknown='error')
X = [[0], [1], [2]]
print(enc.fit(X))
print(enc.categories_)
temp = [[0], [1], [0], [2]]
encoded = enc.transform(temp).toarray().astype(np.float32)
print(encoded)


# enc.inverse_transform([[0, 1, 1, 0, 0], [0, 0, 0, 1, 0]])

feature_names = enc.get_feature_names(['SNP'])
print(feature_names)

In [ ]:
f = open('ADNI_TSV_FILE.raw', 'r')
def generate_raw_snp_samples(line, get_onehot=False):
    line = line.replace('NA', '0').split() 
    FID, IID, PAT,  MAT, SEX, SNPs = line[0], line[1], line[2], line[3], line[4], line[6:]
    SNPs = np.array(SNPs).astype(int)[snps_to_consider]
    if get_onehot:
        SNPs = enc.transform(SNPs.reshape([-1, 1])).toarray().astype(np.float32)
    
    return FID, IID, PAT,  MAT, SEX, SNPs

line = f.readline()
line = f.readline()
FID, IID, PAT,  MAT, SEX, SNPs = generate_raw_snp_samples(line, get_onehot=True)
FID, IID, PAT,  MAT, SEX, SNPs

In [ ]:
f = open('ADNI_TSV_FILE.raw', 'r')

header = f.readline()
open(f'./RAW_SNPs/header.snp', 'w').write(header)

for i in range(847):
    line = f.readline()
    IID = line.split()[1]
    open(f'./RAW_SNPs/{IID}.snp', 'w').write(line)
    
f.close()

In [ ]:
ttl, a, b, c = SNPs.shape[0], SNPs[:, 0].sum(), SNPs[:, 1].sum(), SNPs[:, 2].sum()
ttl, a, b, c, a/ttl, b/ttl, c/ttl, SNPs.shape

In [ ]:
import json
import random
Final_Samples = json.load(open('Final_Samples.json', 'r')) 
negative_samples = Final_Samples[370:]
random.seed(7)
random.shuffle(negative_samples)
Final_Samples = Final_Samples[:370] + negative_samples[:370]
len([x[1] for x in Final_Samples if x[1] == 1]), len([x[1] for x in Final_Samples if x[1] == 0])
Final_Samples = np.array(Final_Samples)
Final_Samples

In [ ]:
Final_Samples = {x[0]:float(x[1]) for x in Final_Samples}
# Final_Samples

In [ ]:
if False:
    def warn(*args, **kwargs):
        pass
    import warnings
    warnings.warn = warn

import os
import time
import sys

import torch
from torch import nn
from torch.autograd import Variable

class simple_model(nn.Module):
    def __init__(self, num_features=1260680, num_hidden=0, hidden_dim=32, drop_probab=.5):
        super(simple_model, self).__init__()
        
        ####
        self.drop_probab = drop_probab
        self.num_hidden = num_hidden
        self.dropout0 = nn.Dropout(p=self.drop_probab)
        self.fc1 = nn.Linear(num_features, hidden_dim)
        self.dropout1 = nn.Dropout(p=self.drop_probab)
        self.fc_hidden = nn.ModuleList([nn.Linear(hidden_dim, hidden_dim) for i in range(self.num_hidden)])
        self.dropout_hidden = nn.ModuleList([nn.Dropout(p=self.drop_probab) for i in range(self.num_hidden)])
        self.fc2 = nn.Linear(hidden_dim, 8)
        self.dropout2 = nn.Dropout(p=self.drop_probab)
        self.outLayer = nn.Linear(8, 1)
        self.sigmoid = nn.Sigmoid()
        self.dropout = nn.functional.dropout
        self.dropout1 = nn.Dropout(p=self.drop_probab)
        ####

    def forward(self, features):
        features = self.dropout0(features)
        features = self.fc1(features)
        features = self.dropout1(features)
        for i in range(self.num_hidden):
            features = self.fc_hidden[i](features)
            features = self.dropout_hidden[i](features)
        features = self.fc2(features)
        features = self.dropout2(features)
        logit = self.outLayer(features)
#         print(features.shape, features)
        probab = self.sigmoid(logit)
        return probab
    

In [ ]:
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

import shap
from copy import deepcopy

model = simple_model(num_features=1260680*3)
model.to(DEVICE)

In [ ]:
# usable_features = torch.autograd.Variable(torch.from_numpy(usable_features)).to(DEVICE).float()

In [ ]:
FID, IID, PAT, MAT, SEX, SNPs = generate_raw_snp_samples(line, get_onehot=True)
SNPs = torch.autograd.Variable(torch.from_numpy(SNPs)).to(DEVICE).float()
FID, IID, PAT, MAT, SEX, SNPs
model.eval()
model(SNPs.view([-1]))